# Homework 2 - Credit Analytics
Submission by Dennis Goldenberg

In [2]:
import numpy as np
#import pandas as pd

## $\underline{\text{Question 1}}$

### a. Expected Value of Loss Leg
I use the formula for expectation and the formulas given to calculate:
$$
\begin{align*}
\mathbb{E}\left[\text{PV}(\text{loss})\right] &= \int_0^{\Delta t}f(t)*\text{PV}\left(\text{loss payment}|\text{default at t}\right)dt\\
&= \int_0^{\Delta t}he^{-ht}*(1 - R)e^{-rt}dt\\
&= h(1 - R)\int_0^{\Delta t}e^{-(r + h)t}dt\\
&= h(1 - R)\left[\frac{e^{-(r + h)t}}{-(r + h)}\right]_0^{\Delta t}\\
&= h(1 - R)\left(\frac{1}{r + h} - \frac{e^{-(r + h)\Delta t}}{r + h}\right)\\
&= \frac{h}{r + h}(1 - R)\left(1 - e^{-(r + h)\Delta t}\right)
\end{align*}
$$
Note that this quantity can be multiplied by the notional amount of the underlying bond; here, the implicit assumption is that the notional amount is 1 dollar.

### b. Expected Value of Refund

I establish a result that will prove useful in the proof:
$$
\int_0^{\Delta t}e^{-(r + h)t}dt = \left[\frac{e^{-(r + h)t}}{-(r + h)}\right]_0^{\Delta t} = \frac{1 - e^{-(r + h)\Delta t}}{r + h} 
$$
Using conditional expectation, I solve:
$$
\begin{align*}
\mathbb{E}\left[\text{PV}\left(\text{Refund}\right)\right] &= \int_0^{\Delta t}\text{PV}\left(\text{Refund}|\text{Default at }t\right)f(t)dt\\
&= \int_0^{\Delta t} P(1 - \frac{t}{\Delta t})e^{-rt} * he^{-ht}dt\\
&= P * h * \int_0^{\Delta t}(1 - \frac{t}{\Delta t})e^{-(r + h)t}dt\\
&= P * h * \left(\int_0^{\Delta t}e^{-(r + h)t}dt - \frac{1}{\Delta t}\int_0^{\Delta t}te^{-(r + h)t}dt\right)\\
&= P * h \left(\frac{1 - e^{-(r + h)\Delta t}}{r + h} - \frac{1}{\Delta t}\int_0^{\Delta t}te^{-(r + h)t}dt\right)
\end{align*}
$$
I then use integration by parts, with $u = t \text{ , } du = dt$  and $dv = e^{-(r + h)t}dt \text{ , } v = \frac{e^{-(r + h)t}}{-(r + h)}$:
$$
\begin{align*}
&\mathbb{E}\left[\text{PV}\left(\text{Refund}\right)\right]\\
&= P * h \left(\frac{1 - e^{-(r + h)\Delta t}}{r + h} - \frac{1}{\Delta t}\int_0^{\Delta t}te^{-(r + h)t}dt\right)\\
&= P * h \left(\frac{1 - e^{-(r + h)\Delta t}}{r + h} - \frac{1}{\Delta t}\left[\frac{t*e^{-(r + h)t}}{-(r + h)} + \frac{1}{r + h}\int_0^{\Delta t}e^{-(r + h)t}dt\right]\right)\\
&= P * h \left(\frac{1 - e^{-(r + h)\Delta t}}{r + h} - \frac{1}{\Delta t}\left[\frac{t*e^{-(r + h)t}}{-(r + h)} + \frac{1}{r + h}*\left(\frac{e^{-(r + h)t}}{-(r + h)}\right)\right]_0^{\Delta t}\right)\\
&= P * h \left(\frac{1 - e^{-(r + h)\Delta t}}{r + h} - \frac{1}{\Delta t}\left[\frac{-\Delta t e^{-(r + h)t}}{r + h} + \frac{1}{(r + h)^2} - \frac{e^{-(r + h)\Delta t}}{(r + h)^2}\right]\right)\\
&= \frac{Ph}{(r + h)}\left(1 - e^{-(r + h)\Delta t} + e^{-(r + h)\Delta t} - \frac{1}{(r + h)\Delta t} + \frac{e^{-(r + h)\Delta t}}{(r + h)\Delta t}\right)\\
&= \frac{Ph}{(r + h)}\left(1 - \frac{1}{(r + h)\Delta t}\left[1 - e^{-(r + h)\Delta t}\right]\right)
\end{align*}
$$

### c. Fair Value of P for One-period Swap
Let $L \sim \text{Exponential}(r + h)$. Therefore, $F_L(\Delta t) = 1 - e^{-(r + h)\Delta t}$. This will be useful for simplifying the final result. I form the break-even premium by recognizing the simple accounting equality:
$$
\text{PV}(\text{Loss}) = \text{PV}(\text{Premium}) - \text{PV}(\text{Refund Payment})
$$
The expected present value of loss is given in part a, and the expected present value of the refund payment is given in part c. Since premium $P$ is paid at the beginning of the preiod, its present value is just $P$. So, I take the expectation of both sides and solve for the break-even premium:
$$
\begin{align*}
&\text{EPV}(\text{Loss}) = P - \text{EPV}(\text{Refund})\\
&\rightarrow \frac{h}{r + h}(1 - R)F_L(\Delta t) = P - \frac{Ph}{r + h}\left(1 - \frac{1}{(r + h)\Delta t}F_L(\Delta t)\right)\\
&\rightarrow \frac{h}{r + h}(1 - R)F_L(\Delta t) = P\left(1 - \frac{h}{r + h}\left(1 - \frac{1}{(r + h)\Delta t}F_L(\Delta t)\right)\right)\\
&\rightarrow P = \frac{\frac{h}{r + h}(1 - R)F_L(\Delta t)}{\left(1 - \frac{h}{r + h}\left(1 - \frac{1}{(r + h)\Delta t}F_L(\Delta t)\right)\right)}
\end{align*}
$$ 
Note that, once again, the notional amount is assumed to be a dollar. So, this break even premium is really the break-even premium per dollar of notional.

## $\underline{\text{Question 2}}$

For this question: read creditportfoliocalculation.pdf

## $\underline{\text{Question 3}}$

For this question: read understandingcopulas.pdf

## $\underline{\text{Question 4}}$

For this question: read AnOverViewofCopulaCDOVerion-6.pdf

## $\underline{\text{Question 5}}$

For this question: read creditportfoliocalculation.pdf

## $\underline{\text{Question 6}}$

For this question: read AnOverViewofCopulaCDOVerion-6.pdf